## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-11-52-35 +0000,nypost,Far-left politician thrown out after interrupt...,https://nypost.com/2025/10/13/us-news/far-left...
1,2025-10-13-11-52-11 +0000,bbc,Lloyds warns car finance scandal could cost it...,https://www.bbc.com/news/articles/cq6zr733917o...
2,2025-10-13-11-51-46 +0000,nyt,Live Updates: Trump Addresses Israeli Parliame...,https://www.nytimes.com/live/2025/10/12/world/...
3,2025-10-13-11-49-22 +0000,bbc,Witkoff praises PM aide's 'tireless efforts' o...,https://www.bbc.com/news/articles/cvg4rwne0ggo...
4,2025-10-13-11-47-57 +0000,bbc,Carmakers go on trial over 'dieselgate' emissi...,https://www.bbc.com/news/articles/cjr5epw8dweo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,38
58,china,18
252,hamas,15
8,israel,15
50,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
237,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...,115
161,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...,105
6,2025-10-13-11-43-22 +0000,wapo,Live updates: Trump declares ‘dawn of a new Mi...,https://www.washingtonpost.com/world/2025/10/1...,103
112,2025-10-13-05-31-15 +0000,nypost,‘War is over’ Trump declares — as he arrives i...,https://nypost.com/2025/10/13/world-news/war-i...,94
85,2025-10-13-08-27-01 +0000,nyt,Asia Markets Fall After Trump Threatens New Ta...,https://www.nytimes.com/2025/10/12/business/tr...,85


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
237,115,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...
112,56,2025-10-13-05-31-15 +0000,nypost,‘War is over’ Trump declares — as he arrives i...,https://nypost.com/2025/10/13/world-news/war-i...
206,50,2025-10-12-17-23-38 +0000,nyt,Cars to Fighter Jets: China’s New Export Curbs...,https://www.nytimes.com/2025/10/12/business/ch...
15,44,2025-10-13-11-28-00 +0000,wsj,"Joel Mokyr, Philippe Aghion and Peter Howitt w...",https://www.wsj.com/economy/nobel-economics-pr...
102,35,2025-10-13-07-05-15 +0000,nypost,Fighting breaks out between Hamas and armed cl...,https://nypost.com/2025/10/13/world-news/fight...
128,31,2025-10-13-03-13-10 +0000,nypost,Wild video shows Texas airport worker lose con...,https://nypost.com/2025/10/12/us-news/wild-vid...
58,29,2025-10-13-09-57-44 +0000,bbc,Forty-two killed as bus crashes on South Afric...,https://www.bbc.com/news/articles/cr5en8325yno...
146,29,2025-10-13-00-21-17 +0000,nypost,Indian student detained by Ukrainian troops wh...,https://nypost.com/2025/10/12/world-news/india...
19,26,2025-10-13-11-13-57 +0000,nypost,President Trump hailed as a ‘colossus’ and ‘gi...,https://nypost.com/2025/10/13/us-news/presiden...
179,25,2025-10-12-20-29-41 +0000,nypost,Washington Post roasts Mamdani’s free NYC bus ...,https://nypost.com/2025/10/12/us-news/washingt...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
